In [1]:
from fastai.vision.all import *

from src.learner import get_learner_task2
from src.utils import (
    load_configuration,
    do_fit
)

# ResNet50 

In [2]:
config  = load_configuration('configs/config1_task2.yml')
learn = get_learner_task2(config)

In [3]:
# learn.lr_find()

In [6]:
do_fit(learn, 'task2_resnet50', epochs=5, lr=1e-3, pct_start=0.75)

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.779966,2.191136,0.583700,0.878500,02:35
1,2.489281,2.078490,0.602400,0.896900,02:12
2,2.348256,2.011742,0.607800,0.904500,02:12
3,2.318217,1.968927,0.619200,0.906600,02:11
4,2.165783,1.925102,0.633100,0.910900,02:09


Better model found at epoch 0 with valid_loss value: 2.191136360168457.
Better model found at epoch 1 with valid_loss value: 2.0784895420074463.
Better model found at epoch 2 with valid_loss value: 2.011742115020752.
Better model found at epoch 3 with valid_loss value: 1.9689271450042725.
Better model found at epoch 4 with valid_loss value: 1.9251015186309814.


# XResNet50 

In [2]:
config  = load_configuration('configs/config2_task2.yml')
learn = get_learner_task2(config)

In [4]:
# do_fit(learn, 'task2_xresnet50', epochs=5, lr=1e-3, pct_start=0.75)